In [ ]:
import requests
import json
import pandas as pd

fdi_key = {
    # Request headers of fund daily info
    'Ocp-Apim-Subscription-Key': 'afbbb604d2814b59b0e5fa6ff3717e16'
}

ffs_key = {
    # Request headers of fund fact sheet
    'Ocp-Apim-Subscription-Key': '16c86c3eb3cb4fee8f54932b92630aef'
}

In [1]:
list_rmf_path = 'C:/Users/momot/Documents/Mahidol/Senior project/Finance-Made-Easy-Personalized-Planner/airflow/config/rmf.txt'
with open(list_rmf_path, 'r') as file:
    list_rmf = [line.strip() for line in file]

print(list_rmf)

['M0049_2565', 'M0074_2544', 'M0079_2544', 'M0080_2544', 'M0081_2544', 'M0088_2565', 'M0146_2565', 'M0181_2564', 'M0236_2554', 'M0295_2564', 'M0310_2564', 'M0317_2565', 'M0374_2565', 'M0424_2562', 'M0425_2562', 'M0426_2562', 'M0457_2565', 'M0511_2558', 'M0518_2558', 'M0522_2558', 'M0547_2564', 'M0548_2564', 'M0555_2564', 'M0570_2560', 'M0593_2558', 'M0612_2558', 'M0721_2565', 'M0778_2554', 'M0787_2554', 'M0933_2554', 'M0935_2554']


In [24]:
from datetime import datetime

date = ''
with open('../config/data/rmf_nav_data/M0074_2544_RMF.csv', "r", encoding='utf-8') as file:
    date = file.readlines()[-1]

temp = date.split(",")
actual_date = temp[0].replace('"', "")
date_object = datetime.strptime(actual_date, '%Y-%m-%d')
date_object

datetime.datetime(2023, 8, 9, 0, 0)

In [14]:
from datetime import date, timedelta
from datetime import datetime

# ไม่ได้ดึงวันนี้ แต่ดึงถึง วันนี้ - 1
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

def convert_string_to_datetime(date_string:str)-> datetime:
    date_object = datetime.strptime(date_string, '%Y-%m-%d')
    return date_object


In [ ]:
for rmf_id in list_rmf:
    year =  rmf_id.rsplit("_", 1) #Split _ to dictate years in Thai Calendar, TH year - 543 = A.D., B Year
    start_year = (int)(year[1])-543 #At index 1, in list String year is Year in TH, then we subtract by 543
    start_date = datetime(start_year,1,1)
    filename = f'../config/data/rmf_nav_data/'+rmf_id+'_RMF.csv'
    # writemode = 'w' เขียนทับ
    with open(filename, 'w',newline = '') as f_out:
        f_out = csv.writer(f_out,quoting=csv.QUOTE_ALL)
        f_out.writerow(header)
        for date in daterange(start_date,end_date):
                res_FDI = requests.get(f'https://api.sec.or.th/FundDailyInfo/{rmf_id}/dailynav/{date.strftime("%Y-%m-%d")}', headers = fdi_key)
                time.sleep(0.5)
                print(res_FDI)
                print(date)
                if(res_FDI.status_code == 200):
                    res_json = json.loads(res_FDI.text)
                    print(res_json)
                    # break
                    try:
                        f_out.writerow([res_json['nav_date'],
                                        res_json['net_asset'],
                                        res_json['last_val'],
                                        res_json['amc_info'][0]['sell_price'],
                                        res_json['amc_info'][0]['buy_price'],
                                        res_json['amc_info'][0]['sell_swap_price'],
                                        res_json['amc_info'][0]['buy_swap_price']],
                                        )
                    except Exception as e:
                         pass   